In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, explode, split, lower, dayofweek, year, hour, row_number
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import networkx as nx

In [4]:
# Set up the Spark session
spark = SparkSession.builder \
    .appName("YelpBusinessAnalysis") \
    .getOrCreate()

In [5]:
# Set HDFS file paths
hdfs_business_path = "hdfs://node-master:9000/user/karim/yelp/business/yelp_academic_dataset_business.json"  # Change this to your HDFS path
hdfs_review_path = "hdfs://node-master:9000/user/karim/yelp/review/yelp_academic_dataset_review.json"  # Change this to your HDFS path
hdfs_user_path = "hdfs://node-master:9000/user/karim/yelp/user/yelp_academic_dataset_user.json"  # Change this to your HDFS path
hdfs_checkin_path = "hdfs://node-master:9000/user/karim/yelp/checkin/yelp_academic_dataset_checkin.json"  # Change this to your HDFS path
hdfs_tip_path = "hdfs://node-master:9000/user/karim/yelp/tip/yelp_academic_dataset_tip.json"

In [6]:
# Load the business data from HDFS
df_business = spark.read.json(hdfs_business_path)

25/03/12 02:44:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
# Top 10 businesses with the highest star rating
top_10_businesses = df_business.orderBy('stars', ascending=False).limit(10)
print("Top 10 businesses with the highest star rating:")
top_10_businesses.show()

Top 10 businesses with the highest star rating:


+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|       662 Durham Rd|                NULL|PSo_C1Sfa13JHjzVN...|Pet Services, Pet...|      Newtown|{8:0-17:0, 8:0-19...|      1|40.2734926|  -74.987496|Indian Walk Veter...|      18940|          15|  5.0|   PA|
|  2226 Frankford Ave|{NULL, NULL, NULL...|Gvi0ZAy2iuB2SHHMi...|Waxing, Hair Remo...| Philadelphia|{10:0-18:0, 0:0-0...|      1| 39.980297| -75.1290209|    